In [ ]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.models.transforms.outcome import Standardize
from botorch.models.transforms.input import Normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition.analytic import LogExpectedImprovement
from botorch.optim import optimize_acqf
from gpytorch.likelihoods import GaussianLikelihood
import gpytorch
import win32com.client
from botorch.models.transforms import Normalize
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.metrics import mean_squared_error, mean_absolute_error
torch.set_default_dtype(torch.float64)
import time
from scipy.stats import qmc
import os
import subprocess


In [ ]:
# Generating inital values for Wall slip parameters

tau_w_range = [0.001, 1]  # critical shear stress
m_range = [0.5, 2]

sampler = qmc.LatinHypercube(d=1)
sample = sampler.random(5)

tau_w_values = qmc.scale(sample, tau_w_range[0], tau_w_range[1])
m_values = qmc.scale(sample, m_range[0], m_range[1])



In [ ]:
Synergy = win32com.client.Dispatch("synergy.Synergy")
Synergy.SetUnits("Metric")
moldflow = "C:\\Program Files\\Autodesk\\Moldflow Insight 2023\\bin"


Synergy.OpenProject("C:\\Users\\oskay\\Documents\\My AMI 2023 Projects\\wall_slip\\wall_slip.mpi")
Project = Synergy.Project
Project.OpenItemByName("60SurfaceCoverage_study", "Study")

# Simulation loop for wall slip parameters
for i in range(5):
    tau_w = tau_w_values[i][0]
    m = m_values[i][0]
    print(f"Simulation {i+1} with tau_w : {tau_w} and m: {m}")

    study_name =f"study_{i+1}"
    Project = Synergy.Project
    
    PropEd = Synergy.PropertyEditor
    Prop = PropEd.FindProperty(10090, 1)
    DVec = Synergy.CreateDoubleArray
    
    DVec.AddDouble(tau_w)
    DVec.AddDouble(m)
    DVec.AddDouble(1e-08)
    DVec.AddDouble(0)
    DVec.AddDouble(0)
    
    Prop.FieldValues(51182, DVec)
          
    PropEd.CommitChanges("Process Conditions")

    StudyDoc = Synergy.StudyDoc
    StudyDoc.SaveAs(study_name)

    p = subprocess.Popen(
    [os.path.join(moldflow, "runstudy.exe"), "C:\\Users\\oskay\\Documents\\My AMI 2023 Projects\\wall_slip\\60Surfacecoverage_study.sdy"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    )
    (output, err) = p.communicate()

# Write the output to a log file
with open("C:\\Users\\oskay\\Documents\\study.log", "w") as file:
    file.write(output.decode("windows-1252").strip())


